In [1]:
import keras
import cv2
import numpy as np
import pandas as pd
import os
import scipy
import tensorflow as tf
from itertools import count
from sklearn.metrics import accuracy_score
from keras.datasets import fashion_mnist
from keras import layers
from keras.layers import Dense, Dropout, Flatten, Activation, Input, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.models import Model
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, CSVLogger
from tqdm import tqdm
from IPython.display import clear_output

Using TensorFlow backend.


In [0]:
BATCH_SIZE = 128
EPOCHS = 9999
IMAGE_SIZE = 28
NUM_CLASSES = 10
NUM_CHANNELS = 1
MODEL_NAME = "FashionMNIST_basleine"
PATH = ""
NR_OF_RUNS = 10

# Preprocess

In [0]:
def preprocess(imgs):
    
    return imgs.reshape(imgs.shape[0], IMAGE_SIZE, IMAGE_SIZE, 1)

In [4]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = preprocess(x_train)
x_test = preprocess(x_test)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [0]:
# Convert class vectors to binary class matrices.
y_trainc = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_testc = keras.utils.to_categorical(y_test, NUM_CLASSES)

In [0]:
x_train_full = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train_full /= 255
x_test /= 255

# Model

In [0]:
def FashionMNISTmodel(imsize, num_classes, num_channels):
    inputs = Input((imsize,imsize,num_channels))
    x = Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', strides = 2)(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size = (2,2), strides=(2,2), padding = "same")(x)
    x = Conv2D(filters=32, kernel_size=(1,1), activation='relu', padding='valid')(x)
    x = Conv2D(filters = 10, kernel_size = (1,1),strides = (1,1), padding = 'valid')(x)
    x = GlobalAveragePooling2D()(x)
    outputs = Activation('softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    optimizer = keras.optimizers.Adam(learning_rate = 1e-04)

    model.compile(loss='categorical_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])
    return model

# Train

In [8]:
for run in range(1, NR_OF_RUNS+1):
    
    # Split the data
    x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_trainc, test_size=0.20, shuffle= True)
    
    models = []
    accuracies = []
    predictions = []
    print(f"\n ===== Train model: Baseline: Run: {run}  =====")
        
    # Set the seeds
    np.random.seed(run*17)
    tf.random.set_seed(run*17)

    # Create directories
    os.makedirs(PATH + MODEL_NAME + f"/{run}/history", exist_ok=True)
    os.makedirs(PATH + MODEL_NAME + f"/{run}/weights", exist_ok=True)
        
    # weight init method
    model = FashionMNISTmodel(IMAGE_SIZE,NUM_CLASSES,NUM_CHANNELS)
            
    #save weights 
    weights_path = PATH + MODEL_NAME + f"/{run}/weights/weights-baseline.h5"
            
    if os.path.exists(weights_path):
        print(f"Skipping training of model: weights exists")
        model.load_weights(weights_path)
    else:
        # initiate early stopping
        es = EarlyStopping(min_delta=0.01, patience=3)
        csv_logger = CSVLogger(PATH + MODEL_NAME + f"/{run}/history/history.csv", separator=';')
        # train
        model.fit(x_train,y_train,
                batch_size = BATCH_SIZE,
                epochs = EPOCHS,
                validation_data = (x_val,y_val),
                shuffle = True,
                callbacks=[es, csv_logger])
        model.save_weights(weights_path)
            
    acc = model.evaluate(x_test,y_testc)[1]
    print(f"Run: {run} added. Resulting score: {acc}")

    print("\n ===== Saving results =====")  
    # Save the results
    file = PATH + MODEL_NAME + f"/results_.csv"
    df = pd.DataFrame([[run,acc]])
    if not os.path.isfile(file):
        df.to_csv(file, header=["run","accuracy"], index=False)
    else: # else it exists so append without writing the header
        df.to_csv(file, mode='a', header=False, index=False)
    clear_output(wait=True)


 ===== Train model: Baseline: Run: 10  =====
Train on 48000 samples, validate on 12000 samples
Epoch 1/9999
48000/48000 [==============================] - 3s 54us/step - loss: 2.1949 - accuracy: 0.2062 - val_loss: 2.1966 - val_accuracy: 0.2674
Epoch 2/9999
48000/48000 [==============================] - 2s 47us/step - loss: 1.8949 - accuracy: 0.4039 - val_loss: 1.7785 - val_accuracy: 0.4655
Epoch 3/9999
48000/48000 [==============================] - 3s 53us/step - loss: 1.6435 - accuracy: 0.5035 - val_loss: 1.5341 - val_accuracy: 0.5398
Epoch 4/9999
48000/48000 [==============================] - 2s 47us/step - loss: 1.4610 - accuracy: 0.5590 - val_loss: 1.3856 - val_accuracy: 0.5786
Epoch 5/9999
48000/48000 [==============================] - 2s 47us/step - loss: 1.3382 - accuracy: 0.5927 - val_loss: 1.2831 - val_accuracy: 0.6087
Epoch 6/9999
48000/48000 [==============================] - 2s 49us/step - loss: 1.2490 - accuracy: 0.6139 - val_loss: 1.2074 - val_accuracy: 0.6270
Epoch 7/99

In [9]:
!zip -r /content/FashionMNIST_basleine.zip /content/FashionMNIST_basleine

  adding: content/FashionMNIST_basleine/ (stored 0%)
  adding: content/FashionMNIST_basleine/9/ (stored 0%)
  adding: content/FashionMNIST_basleine/9/weights/ (stored 0%)
  adding: content/FashionMNIST_basleine/9/weights/weights-baseline.h5 (deflated 71%)
  adding: content/FashionMNIST_basleine/9/history/ (stored 0%)
  adding: content/FashionMNIST_basleine/9/history/history.csv (deflated 53%)
  adding: content/FashionMNIST_basleine/7/ (stored 0%)
  adding: content/FashionMNIST_basleine/7/weights/ (stored 0%)
  adding: content/FashionMNIST_basleine/7/weights/weights-baseline.h5 (deflated 70%)
  adding: content/FashionMNIST_basleine/7/history/ (stored 0%)
  adding: content/FashionMNIST_basleine/7/history/history.csv (deflated 52%)
  adding: content/FashionMNIST_basleine/8/ (stored 0%)
  adding: content/FashionMNIST_basleine/8/weights/ (stored 0%)
  adding: content/FashionMNIST_basleine/8/weights/weights-baseline.h5 (deflated 70%)
  adding: content/FashionMNIST_basleine/8/history/ (stored